<a href="https://colab.research.google.com/github/annisafitribas/ft_credit_home/blob/main/ft_credit_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PERSIAPAN**

In [2]:
import os, sys, time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

WORKDIR = '/content/home_credit_task'
os.makedirs(WORKDIR, exist_ok=True)
print("WORKDIR:", WORKDIR)

WORKDIR: /content/home_credit_task


Bertujuan untuk menyiapkan folder kerja dan library yang akan digunakan dalam proses pengolahan data

In [3]:
# install dependencies if missing (Colab-friendly)
try:
    import gdown
except Exception:
    !pip install -q gdown
    import gdown

try:
    import lightgbm as lgb
except Exception:
    !pip install -q lightgbm
    import lightgbm as lgb

try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
except Exception:
    !pip install -q python-pptx
    from pptx import Presentation
    from pptx.util import Inches, Pt

try:
    import matplotlib
    import matplotlib.pyplot as plt
except Exception:
    !pip install -q matplotlib
    import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 10.4 MB/s eta 0:00:00


In [4]:
# sklearn / joblib
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.impute import SimpleImputer
import joblib
import pandas as pd
import numpy as np
from pathlib import Path

# 1. Download dataset file

In [5]:
files = {
    'application_train.csv': '1q059QolR6CNxB0PWESAjkEWIprNutajA',
    'application_test.csv' : '1QD7ehk_hzXze0vHQuYa5qyqfDcfI8Sex',
    'bureau.csv'           : '1hndizX1t5ab0DTnKMTedqVJ1ZxLVclhF',
    'bureau_balance.csv'   : '1OXEQb_L6S_mZALJi4--C6RyFI6yOsq4x',
    'credit_card_balance.csv': '1t6Hhsmj0vSCCKUlNXht_xDQ6Z6l4M0Vu',
    'installments_payments.csv': '126xrKCW5EQrxkQoDwmN-yb00ILBKnhR8',
    'POS_CASH_balance.csv' : '1dODAmBQLaylpM2JcCHfc4KNbbtKY7xhA',
    'previous_application.csv': '1D4O7xf-lF_3oBeu6XMwzhpXtSvhcgoBU',
    'HomeCredit_columns_description.csv': '1v2iGGOJjlUGSTsQz-bsjtjtyM5IQp7uW',
    'sample_submission.csv': '1JongVA9fWMYml5XKVnbhm8TUlR5Efs0n'
}

for fname, fid in files.items():
    dest = os.path.join(WORKDIR, fname)
    if not os.path.exists(dest):
        print("Downloading", fname)
        url = f"https://drive.google.com/uc?export=download&id={fid}"
        gdown.download(url, dest, quiet=False)
    else:
        print("Exists:", fname)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1q059QolR6CNxB0PWESAjkEWIprNutajA
From (redirected): https://drive.google.com/uc?export=download&id=1q059QolR6CNxB0PWESAjkEWIprNutajA&confirm=t&uuid=25d94ba2-580e-4722-9845-3773dec0d1a0
To: /content/home_credit_task/application_train.csv
100%|██████████| 166M/166M [00:02<00:00, 72.4MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1QD7ehk_hzXze0vHQuYa5qyqfDcfI8Sex
To: /content/home_credit_task/application_test.csv
100%|██████████| 26.6M/26.6M [00:00<00:00, 48.4MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1hndizX1t5ab0DTnKMTedqVJ1ZxLVclhF
From (redirected): https://drive.google.com/uc?export=download&id=1hndizX1t5ab0DTnKMTedqVJ1ZxLVclhF&confirm=t&uuid=fd5b9554-2f13-456d-98ef-610362456f5d
To: /content/home_credit_task/bureau.csv
100%|██████████| 170M/170M [00:02<00:00, 60.6MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1OXEQb_L6S_mZALJi4--C6RyFI6yOsq4x
From (redirected): https://drive.google.com/uc?export=download&id=1OXEQb_L6S_mZALJi4--C6RyFI6yOsq4x&confirm=t&uuid=935b25ee-d8e6-4458-9c06-bc5244cad164
To: /content/home_credit_task/bureau_balance.csv
100%|██████████| 376M/376M [00:05<00:00, 73.2MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1t6Hhsmj0vSCCKUlNXht_xDQ6Z6l4M0Vu
From (redirected): https://drive.google.com/uc?export=download&id=1t6Hhsmj0vSCCKUlNXht_xDQ6Z6l4M0Vu&confirm=t&uuid=218e4c45-ebd9-4412-bfb4-d43c25b8c8fa
To: /content/home_credit_task/credit_card_balance.csv
100%|██████████| 425M/425M [00:07<00:00, 56.6MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=126xrKCW5EQrxkQoDwmN-yb00ILBKnhR8
From (redirected): https://drive.google.com/uc?export=download&id=126xrKCW5EQrxkQoDwmN-yb00ILBKnhR8&confirm=t&uuid=ec9a5be1-638f-4d73-8481-ac6fc7eee529
To: /content/home_credit_task/installments_payments.csv
100%|██████████| 723M/723M [00:08<00:00, 80.7MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1dODAmBQLaylpM2JcCHfc4KNbbtKY7xhA
From (redirected): https://drive.google.com/uc?export=download&id=1dODAmBQLaylpM2JcCHfc4KNbbtKY7xhA&confirm=t&uuid=334eba1d-79bb-47b2-bf52-023242f20f1e
To: /content/home_credit_task/POS_CASH_balance.csv
100%|██████████| 393M/393M [00:04<00:00, 81.4MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1D4O7xf-lF_3oBeu6XMwzhpXtSvhcgoBU
From (redirected): https://drive.google.com/uc?export=download&id=1D4O7xf-lF_3oBeu6XMwzhpXtSvhcgoBU&confirm=t&uuid=b19155fb-402f-49e2-b498-4ec7a50b2ec6
To: /content/home_credit_task/previous_application.csv
100%|██████████| 405M/405M [00:05<00:00, 76.2MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1v2iGGOJjlUGSTsQz-bsjtjtyM5IQp7uW
To: /content/home_credit_task/HomeCredit_columns_description.csv
100%|██████████| 37.4k/37.4k [00:00<00:00, 17.4MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1JongVA9fWMYml5XKVnbhm8TUlR5Efs0n
To: /content/home_credit_task/sample_submission.csv
100%|██████████| 536k/536k [00:00<00:00, 7.22MB/s]


# Load csv

In [ ]:
print("\nLoading CSVs ...")
train = pd.read_csv(os.path.join(WORKDIR, 'application_train.csv'), low_memory=False)
test  = pd.read_csv(os.path.join(WORKDIR, 'application_test.csv'), low_memory=False)
bureau = pd.read_csv(os.path.join(WORKDIR, 'bureau.csv'), low_memory=False)
bureau_balance = pd.read_csv(os.path.join(WORKDIR, 'bureau_balance.csv'), low_memory=False)
credit_card_balance = pd.read_csv(os.path.join(WORKDIR, 'credit_card_balance.csv'), low_memory=False)
installments = pd.read_csv(os.path.join(WORKDIR, 'installments_payments.csv'), low_memory=False)
pos_cash = pd.read_csv(os.path.join(WORKDIR, 'POS_CASH_balance.csv'), low_memory=False)
prev_app = pd.read_csv(os.path.join(WORKDIR, 'previous_application.csv'), low_memory=False)
sample_sub = pd.read_csv(os.path.join(WORKDIR, 'sample_submission.csv'), low_memory=False)

print("Shapes:")
print("train", train.shape, "test", test.shape)
print("bureau", bureau.shape, "bureau_balance", bureau_balance.shape)
print("credit_card_balance", credit_card_balance.shape, "installments", installments.shape)
print("pos_cash", pos_cash.shape, "previous_application", prev_app.shape)


Loading CSVs ...


# **3. Quick EDA & visuals**

In [ ]:
print("\n--- TARGET distribution ---")
print(train['TARGET'].value_counts(normalize=True))

# helper for saving plots
def savefig(fig, filename):
    path = os.path.join(WORKDIR, filename)
    fig.savefig(path, bbox_inches='tight')
    print('Saved plot:', path)

## 3.1 target distribution plot

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
counts = train['TARGET'].value_counts().sort_index()
ax.bar(counts.index.astype(str), counts.values)
ax.set_title('Target distribution (counts)')
ax.set_xlabel('TARGET')
ax.set_ylabel('Count')
savefig(fig, 'target_distribution.png')
plt.close(fig)

## 3.2 top missing features (bar)

In [ ]:
missing = train.isna().mean().sort_values(ascending=False).head(30)
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.barh(missing.index[::-1], missing.values[::-1])
ax.set_title('Top missing percentage (train)')
ax.set_xlabel('Fraction missing')
savefig(fig, 'missing_pct_top.png')
plt.close(fig)

## 3.3 correlation heatmap of numeric features (sampled for speed)

In [ ]:
num = train.select_dtypes(include=[np.number]).drop(['SK_ID_CURR','TARGET'], axis=1, errors='ignore')
# sample columns to avoid huge matrix
num_small = num.sample(n=min(30, num.shape[1]), axis=1)
corr = num_small.corr()
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
cax = ax.imshow(corr.values, interpolation='nearest')
ax.set_xticks(np.arange(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90, fontsize=8)
ax.set_yticks(np.arange(len(corr.columns)))
ax.set_yticklabels(corr.columns, fontsize=8)
ax.set_title('Correlation matrix (subset)')
fig.colorbar(cax, ax=ax)
savefig(fig, 'corr_matrix_subset.png')
plt.close(fig)

# **4. Feature engineering (same as baseline but kept clear)**

## 4.1 Aggregations dari bureau (per SK_ID_CURR)

In [ ]:
b_agg = bureau.groupby('SK_ID_CURR').agg(
    bureau_loans_count = ('SK_ID_BUREAU', 'count'),
    bureau_credit_sum_mean = ('AMT_CREDIT_SUM', 'mean'),
    bureau_credit_sum_max = ('AMT_CREDIT_SUM', 'max'),
    bureau_active_cnt = ('CREDIT_ACTIVE', lambda x: (x=='Active').sum())
).reset_index()

## 4.2 bureau_balance -> bad rate per bureau id then agg

In [ ]:
bb_bad = bureau_balance[bureau_balance['STATUS'].isin(['2','3','4','5'])].groupby('SK_ID_BUREAU').size().rename('bad_months')
bb_tot = bureau_balance.groupby('SK_ID_BUREAU').size().rename('total_months')
bb = pd.concat([bb_bad, bb_tot], axis=1).fillna(0)
bb['bad_rate'] = bb['bad_months'] / bb['total_months']
bureau2 = bureau.merge(bb.reset_index(), on='SK_ID_BUREAU', how='left')
b2_agg = bureau2.groupby('SK_ID_CURR').agg(
    bureau_prev_bad_rate_mean = ('bad_rate','mean'),
    bureau_prev_months_mean = ('total_months','mean')
).reset_index()

b_agg = b_agg.merge(b2_agg, on='SK_ID_CURR', how='left')

## 4.3 previous_application aggregates

In [ ]:
prev_agg = prev_app.groupby('SK_ID_CURR').agg(
    prev_count = ('SK_ID_PREV','count'),
    prev_amt_app_mean = ('AMT_APPLICATION','mean'),
    prev_amt_credit_mean = ('AMT_CREDIT','mean'),
    prev_approved = ('NAME_CONTRACT_STATUS', lambda x: (x=='Approved').sum())
).reset_index()

## # 4.4 installments

In [ ]:
inst_agg = installments.groupby('SK_ID_CURR').agg(
    inst_count = ('NUM_INSTALMENT_VERSION','count'),
    inst_amt_sum = ('AMT_PAYMENT','sum'),
    inst_delay_mean = ('DAYS_ENTRY_PAYMENT', lambda x: np.nanmean(x - installments.loc[x.index,'DAYS_INSTALMENT']))
).reset_index()

## 4.5 credit_card & pos

In [ ]:
cc_agg = credit_card_balance.groupby('SK_ID_CURR').agg(
    cc_count = ('SK_ID_PREV','count'),
    cc_bal_mean = ('AMT_BALANCE','mean'),
    cc_limit_mean = ('AMT_CREDIT_LIMIT_ACTUAL','mean')
).reset_index()

pos_agg = pos_cash.groupby('SK_ID_CURR').agg(
    pos_count = ('SK_ID_PREV','count'),
    pos_dpd_mean = ('SK_DPD','mean')
).reset_index()

## 4.6 application-level features

In [ ]:
def make_app_features(df):
    df = df.copy()
    df['DAYS_BIRTH_YEARS'] = (-df['DAYS_BIRTH']) / 365.25
    df['DAYS_EMPLOYED_YEARS'] = df['DAYS_EMPLOYED'].replace(365243, np.nan) / -365.25
    df['INCOME_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / (df['AMT_CREDIT'] + 1)
    df['CREDIT_GOODS_RATIO'] = df['AMT_CREDIT'] / (df['AMT_GOODS_PRICE'] + 1)
    return df[['SK_ID_CURR','DAYS_BIRTH_YEARS','DAYS_EMPLOYED_YEARS','AMT_INCOME_TOTAL','AMT_CREDIT','INCOME_CREDIT_RATIO','CREDIT_GOODS_RATIO']]

app_train_feats = make_app_features(train)
app_test_feats = make_app_features(test)

# **5. Merge features**

In [ ]:
train_base = train[['SK_ID_CURR','TARGET']].merge(b_agg, on='SK_ID_CURR', how='left') \
                               .merge(prev_agg, on='SK_ID_CURR', how='left') \
                               .merge(inst_agg, on='SK_ID_CURR', how='left') \
                               .merge(cc_agg, on='SK_ID_CURR', how='left') \
                               .merge(pos_agg, on='SK_ID_CURR', how='left') \
                               .merge(app_train_feats, on='SK_ID_CURR', how='left')

test_base = test[['SK_ID_CURR']].merge(b_agg, on='SK_ID_CURR', how='left') \
                               .merge(prev_agg, on='SK_ID_CURR', how='left') \
                               .merge(inst_agg, on='SK_ID_CURR', how='left') \
                               .merge(cc_agg, on='SK_ID_CURR', how='left') \
                               .merge(pos_agg, on='SK_ID_CURR', how='left') \
                               .merge(app_test_feats, on='SK_ID_CURR', how='left')

# **6. Prepare X, y; preprocessing**

In [ ]:
Y = train_base['TARGET']
X = train_base.drop(['SK_ID_CURR','TARGET'], axis=1)
X_test = test_base.drop(['SK_ID_CURR'], axis=1)

# numeric / categorical separation
num_cols = [c for c in X.columns if X[c].dtype.kind in 'biufc']
cat_cols = [c for c in X.columns if c not in num_cols]

# Impute numeric with median (computed from train)
num_imputer = SimpleImputer(strategy='median')
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

# For categorical (if any), fill and label-encode simple
for c in cat_cols:
    X[c] = X[c].fillna('MISSING').astype(str)
    X_test[c] = X_test[c].fillna('MISSING').astype(str)
for c in cat_cols:
    X[c], _ = pd.factorize(X[c])
    X_test[c], _ = pd.factorize(X_test[c])

# Ensure X_test has all columns (reindex)
X_test = X_test.reindex(columns=X.columns, fill_value=0)

print("\nNumber of features:", X.shape[1])

# **7. Train/validation split and scaling**

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)
scaler = StandardScaler()
X_tr_scaled = scaler.fit_transform(X_tr)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# **8. Logistic Regression**

In [ ]:
print("\n=== Logistic Regression ===")
lr = LogisticRegression(max_iter=2000, class_weight='balanced', n_jobs=-1)
# 5-fold CV
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
lr_cv_scores = []
for tr_idx, vc_idx in kf.split(X, Y):
    lr.fit(scaler.fit_transform(X.iloc[tr_idx]), Y.iloc[tr_idx])
    p = lr.predict_p#roba(scaler.transform(X.iloc[vc_idx]))[:,1]
    lr_cv_scores.append(roc_auc_score(Y.iloc[vc_idx], p))
print("LR 5-fold AUC: %.5f ± %.5f" % (np.mean(lr_cv_scores), np.std(lr_cv_scores)))

# fit on training partition and validate
lr.fit(X_tr_scaled, y_tr)
proba_lr_val = lr.predict_proba(X_val_scaled)[:,1]
print("LR holdout AUC:", roc_auc_score(y_val, proba_lr_val))
print("LR classification report (holdout, threshold=0.5):")
print(classification_report(y_val, (proba_lr_val>0.5).astype(int)))

# save logistic model & scaler
joblib.dump(lr, os.path.join(WORKDIR, 'model_logistic.pkl'))
joblib.dump(scaler, os.path.join(WORKDIR, 'scaler_logistic.pkl'))

# ROC curve plot (LR)
fpr, tpr, _ = roc_curve(y_val, proba_lr_val)
roc_auc_lr = auc(fpr, tpr)
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
ax.plot(fpr, tpr)
ax.plot([0,1],[0,1], linestyle='--')
ax.set_title(f'Logistic ROC (AUC={roc_auc_lr:.4f})')
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
savefig(fig, 'roc_logistic.png')
plt.close(fig)

# **9. LightGBM with RandomizedSearchCV**

In [ ]:
print("\n=== LightGBM (RandomizedSearchCV) ===")
from scipy.stats import randint as sp_randint, uniform as sp_uniform

lgb_clf = lgb.LGBMClassifier(objective='binary', random_state=42, n_jobs=-1, verbosity=-1)

param_dist = {
    'num_leaves': [31, 63, 127],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'n_estimators': [200, 400, 800],
    'min_child_samples': [5, 20, 50],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

rs = RandomizedSearchCV(lgb_clf, param_distributions=param_dist, n_iter=12, scoring='roc_auc', cv=3, random_state=42, n_jobs=-1, verbose=1)
rs.fit(X, Y)
print("Best LGB params:", rs.best_params_)
print("Best LGB CV score:", rs.best_score_)

best_lgb = rs.best_estimator_
# evaluate on holdout
best_lgb.fit(X_tr, y_tr)
proba_lgb_val = best_lgb.predict_proba(X_val)[:,1]
print("LGB holdout AUC:", roc_auc_score(y_val, proba_lgb_val))
print("LGB classification report (holdout):")
print(classification_report(y_val, (proba_lgb_val>0.5).astype(int)))

# Save LGB model
joblib.dump(best_lgb, os.path.join(WORKDIR, 'model_lgb.pkl'))

# ROC curve plot (LGB)
fpr_l, tpr_l, _ = roc_curve(y_val, proba_lgb_val)
roc_auc_lgb = auc(fpr_l, tpr_l)
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
ax.plot(fpr_l, tpr_l)
ax.plot([0,1],[0,1], linestyle='--')
ax.set_title(f'LightGBM ROC (AUC={roc_auc_lgb:.4f})')
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
savefig(fig, 'roc_lgb.png')
plt.close(fig)

# Combined ROC
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
ax.plot(fpr, tpr)
ax.plot(fpr_l, tpr_l)
ax.plot([0,1],[0,1], linestyle='--')
ax.set_title(f'ROC comparison (LR {roc_auc_lr:.4f} vs LGB {roc_auc_lgb:.4f})')
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
savefig(fig, 'roc_comparison.png')
plt.close(fig)

# **10. Feature importance (LGB) and plot**

In [ ]:
fi = pd.Series(best_lgb.feature_importances_, index=X.columns).sort_values(ascending=False)
print('\nTop 20 features (LightGBM):')
print(fi.head(20))

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
topn = fi.head(20)[::-1]
ax.barh(topn.index, topn.values)
ax.set_title('Top 20 feature importances (LightGBM)')
ax.set_xlabel('Importance')
savefig(fig, 'feature_importance_top20.png')
plt.close(fig)
